VGG

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 資料前處理 -----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 模型定義 -----------
class VGG16Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        base = models.vgg16_bn(pretrained=True)
        self.features = base.features
        for param in self.features.parameters():
            param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

# ----------- 訓練與測試函式 -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None
    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping()

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg = VGG16Transfer()
model_vgg = train(model_vgg, train_loader, val_loader, device)
test_acc = evaluate(model_vgg, test_loader, device)
print(f"🎯 VGG16 Test Accuracy: {test_acc:.2f}%")


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\PC/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:51<00:00, 10.7MB/s] 


[Epoch 1] Train Acc: 69.89% | Val Acc: 65.91%
[Epoch 2] Train Acc: 98.30% | Val Acc: 84.09%
[Epoch 3] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 4] Train Acc: 100.00% | Val Acc: 90.91%
[Epoch 5] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 6] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 7] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 8] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 9] Train Acc: 100.00% | Val Acc: 88.64%
⛔ Early stopping
🎯 VGG16 Test Accuracy: 83.75%


微調-凍結前50層

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 資料前處理 -----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 模型定義（微調版）-----------
class VGG16Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        base = models.vgg16_bn(pretrained=True)
        self.features = base.features

        # ✅ 凍結前 33 層，只微調最後幾層
        for param in self.features[:50]:
            param.requires_grad = False
        for param in self.features[50:]:
            param.requires_grad = True

        # 分類器定義
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

# ----------- 訓練與測試函式 -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None
    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping()

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg = VGG16Transfer()
model_vgg = train(model_vgg, train_loader, val_loader, device)
test_acc = evaluate(model_vgg, test_loader, device)
print(f"🎯 VGG16 Test Accuracy: {test_acc:.2f}%")


[Epoch 1] Train Acc: 66.48% | Val Acc: 72.73%
[Epoch 2] Train Acc: 99.43% | Val Acc: 86.36%
[Epoch 3] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 4] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 5] Train Acc: 100.00% | Val Acc: 79.55%
[Epoch 6] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 7] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 8] Train Acc: 100.00% | Val Acc: 86.36%
⛔ Early stopping
🎯 VGG16 Test Accuracy: 86.25%


微調-凍結前 33 層

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 資料前處理 -----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 模型定義（微調版）-----------
class VGG16Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        base = models.vgg16_bn(pretrained=True)
        self.features = base.features

        # ✅ 凍結前 33 層，只微調最後幾層
        for param in self.features[:33]:
            param.requires_grad = False
        for param in self.features[33:]:
            param.requires_grad = True

        # 分類器定義
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

# ----------- 訓練與測試函式 -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None
    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping()

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg = VGG16Transfer()
model_vgg = train(model_vgg, train_loader, val_loader, device)
test_acc = evaluate(model_vgg, test_loader, device)
print(f"🎯 VGG16 Test Accuracy: {test_acc:.2f}%")


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1] Train Acc: 68.18% | Val Acc: 61.36%
[Epoch 2] Train Acc: 92.05% | Val Acc: 68.18%
[Epoch 3] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 4] Train Acc: 100.00% | Val Acc: 70.45%
[Epoch 5] Train Acc: 100.00% | Val Acc: 70.45%
[Epoch 6] Train Acc: 100.00% | Val Acc: 70.45%
[Epoch 7] Train Acc: 100.00% | Val Acc: 79.55%
[Epoch 8] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 9] Train Acc: 100.00% | Val Acc: 90.91%
[Epoch 10] Train Acc: 100.00% | Val Acc: 93.18%
[Epoch 11] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 12] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 13] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 14] Train Acc: 100.00% | Val Acc: 81.82%
[Epoch 15] Train Acc: 100.00% | Val Acc: 79.55%
⛔ Early stopping
🎯 VGG16 Test Accuracy: 87.50%


微調-凍結前15層

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 資料前處理 -----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 模型定義（微調版）-----------
class VGG16Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        base = models.vgg16_bn(pretrained=True)
        self.features = base.features

        # ✅ 凍結後 33 層，只微調最後幾層
        for param in self.features[:15]:
            param.requires_grad = False
        for param in self.features[15:]:
            param.requires_grad = True

        # 分類器定義
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

# ----------- 訓練與測試函式 -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None
    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping()

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg = VGG16Transfer()
model_vgg = train(model_vgg, train_loader, val_loader, device)
test_acc = evaluate(model_vgg, test_loader, device)
print(f"🎯 VGG16 Test Accuracy: {test_acc:.2f}%")


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1] Train Acc: 71.02% | Val Acc: 68.18%
[Epoch 2] Train Acc: 98.30% | Val Acc: 75.00%
[Epoch 3] Train Acc: 99.43% | Val Acc: 79.55%
[Epoch 4] Train Acc: 100.00% | Val Acc: 79.55%
[Epoch 5] Train Acc: 100.00% | Val Acc: 79.55%
[Epoch 6] Train Acc: 100.00% | Val Acc: 81.82%
[Epoch 7] Train Acc: 100.00% | Val Acc: 81.82%
[Epoch 8] Train Acc: 100.00% | Val Acc: 81.82%
[Epoch 9] Train Acc: 100.00% | Val Acc: 77.27%
[Epoch 10] Train Acc: 100.00% | Val Acc: 77.27%
[Epoch 11] Train Acc: 100.00% | Val Acc: 77.27%
⛔ Early stopping
🎯 VGG16 Test Accuracy: 86.25%


微調-凍結前10層

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 資料前處理 -----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 模型定義（微調版）-----------
class VGG16Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        base = models.vgg16_bn(pretrained=True)
        self.features = base.features

        # ✅ 凍結後 33 層，只微調最後幾層
        for param in self.features[:10]:
            param.requires_grad = False
        for param in self.features[10:]:
            param.requires_grad = True

        # 分類器定義
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

# ----------- 訓練與測試函式 -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None
    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping()

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg = VGG16Transfer()
model_vgg = train(model_vgg, train_loader, val_loader, device)
test_acc = evaluate(model_vgg, test_loader, device)
print(f"🎯 VGG16 Test Accuracy: {test_acc:.2f}%")


[Epoch 1] Train Acc: 60.23% | Val Acc: 81.82%
[Epoch 2] Train Acc: 98.30% | Val Acc: 86.36%
[Epoch 3] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 4] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 5] Train Acc: 100.00% | Val Acc: 90.91%
[Epoch 6] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 7] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 8] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 9] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 10] Train Acc: 100.00% | Val Acc: 88.64%
⛔ Early stopping
🎯 VGG16 Test Accuracy: 86.25%
